## 准备数据

In [31]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [32]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [33]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        initializer = tf.keras.initializers.GlorotNormal()

        self.W1 = tf.Variable(initializer([28*28, 1024]), dtype=tf.float32)
        self.b1 = tf.Variable(tf.zeros([1024]), dtype=tf.float32)
        self.W2 = tf.Variable(initializer([1024, 10]), dtype=tf.float32)
        self.b2 = tf.Variable(tf.zeros([10]), dtype=tf.float32)

        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [34]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [35]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3814514 ; accuracy 0.13141666
epoch 1 : loss 2.3446338 ; accuracy 0.14243333
epoch 2 : loss 2.3105402 ; accuracy 0.15438333
epoch 3 : loss 2.2785783 ; accuracy 0.16863333
epoch 4 : loss 2.2483296 ; accuracy 0.18456666
epoch 5 : loss 2.2194908 ; accuracy 0.20193334
epoch 6 : loss 2.1918364 ; accuracy 0.22045
epoch 7 : loss 2.1651974 ; accuracy 0.24095
epoch 8 : loss 2.1394427 ; accuracy 0.26368332
epoch 9 : loss 2.1144702 ; accuracy 0.28476667
epoch 10 : loss 2.0902004 ; accuracy 0.30616668
epoch 11 : loss 2.0665684 ; accuracy 0.3279
epoch 12 : loss 2.0435228 ; accuracy 0.35033333
epoch 13 : loss 2.0210214 ; accuracy 0.37103334
epoch 14 : loss 1.999029 ; accuracy 0.39323333
epoch 15 : loss 1.9775169 ; accuracy 0.41371667
epoch 16 : loss 1.9564607 ; accuracy 0.43423334
epoch 17 : loss 1.9358399 ; accuracy 0.45446667
epoch 18 : loss 1.9156368 ; accuracy 0.47376665
epoch 19 : loss 1.8958356 ; accuracy 0.49193335
epoch 20 : loss 1.8764236 ; accuracy 0.50918335
epoch 21 : lo